In [ ]:
import os
import torchvision.utils as utils
import sys

sys.path.append("../")
from model.models import *
from model.cond_refinenet_dilated import *
from model.ema import EMAHelper

from functions.loss import *
from functions.utils import *

import dataset
import itertools
import matplotlib.pyplot as plt

Does not make sense to perform PCA using diagonal matrix parameterized by UNet
but can visualize diagonal and different noise scale directly

In [ ]:
from main import dict2namespace
import yaml

sigma = 0.5
with open("../runs/logs/diagonal_cifar10_0.5/config.yml".format(sigma), 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
config = dict2namespace(config)

In [ ]:
dim = 3 * 32 * 32
config.device = device = "cuda"
ckpt_path = "../runs/logs/diagonal_cifar10_0.5/ckpt".format(sigma)

state_dict = torch.load(os.path.join(ckpt_path, 'checkpoint.pth'), map_location=device)
model = CondRefineNetDilated(config).to(device)
model = torch.nn.DataParallel(model)
model.load_state_dict(state_dict[0])
 
# load s1 model
s1 = CondRefineNetDilated(config).to(device)
s1 = torch.nn.DataParallel(s1)
s1.load_state_dict(state_dict[1])

if config.model.ema:
    ema_helper = EMAHelper(mu=config.model.ema_rate)
    ema_helper.register(model)
    ema_helper_s1 = EMAHelper(mu=config.model.ema_rate)
    ema_helper_s1.register(s1)

if config.model.ema:
    ema_helper.load_state_dict(state_dict[3])
    ema_helper_s1.load_state_dict(state_dict[4])
    
if config.model.ema:
    model = ema_helper.ema_copy(model)
    s1 = ema_helper_s1.ema_copy(s1)
else:
    model = model
    s1 = s1
    
model.eval()
s1.eval()
print("model loaded")

In [ ]:
import torchvision
import torchvision.transforms as transforms
transform = transforms.Compose([transforms.ToTensor(),])

batch_size = 36
channel = 3
image_size = img = 32

testset = torchvision.datasets.CIFAR10(root='../data/cifar10', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=True, num_workers=2)


In [ ]:
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
dataiter = iter(testloader)
test_X, _ = dataiter.next()

In [ ]:
channel = 3
perturb_X = test_X * 2. - 1.
perturb_X = perturb_X + torch.randn_like(test_X) * sigma
perturb_X = perturb_X.view(perturb_X.shape[0], -1)
with torch.no_grad():
    score_2 = model(perturb_X).detach().reshape(perturb_X.shape[0], channel, image_size, image_size)
    covariance = score_2 * (sigma ** 4) + 1. * (sigma ** 2)
    print(covariance.shape)

In [ ]:
cov_diagonal = covariance.clone()
cov_diagonal.max(), cov_diagonal.min()

In [ ]:
imshow(torchvision.utils.make_grid(test_X.to("cpu"), nrow=6))

In [ ]:
cov_channel = 1
cov_ = cov_diagonal[:,cov_channel,:,:]
img = torchvision.utils.make_grid(cov_, nrow=6, padding=2, pad_value=-1).to("cpu") # 3 channels are the same, only take one of it
img = img.reshape(6, 6, 32, 32).permute(0, 2, 1, 3).reshape(6*32, 6*32)
npimg = img.numpy()
plt.imshow(npimg, vmin=-0., vmax=0.1)

In [ ]:
index = 0
plt.axis('off')
imshow(torchvision.utils.make_grid(test_X[index].to("cpu")))

In [ ]:
plt.axis('off')
imshow(torchvision.utils.make_grid((perturb_X[index]*0.5+0.5).reshape(3, 32, 32).to("cpu")))

In [ ]:
cov_diagonal[index,1].cpu().max(), cov_diagonal[index,0].cpu().min()

In [ ]:
plt.axis('off')
plt.imshow(cov_diagonal[index,cov_channel].cpu().reshape(32, 32).numpy(), vmin=-0., vmax=0.1)